<a href="https://colab.research.google.com/github/mtracewicz/DiplomaSeminar/blob/master/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-NET

## Import

In [1]:
import sys
import tensorflow as tf

## Hyperparameters

In [2]:
learning_rate = 0.01
epochs = 40
batch_size = 4000
validation_split = 0.2

## Args

In [3]:
checkpoint_name = "unet"

## Create model

In [4]:
def create_model(learning_rate):
    # Model creation
    model = tf.keras.models.Sequential()
    # Downsampling
    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2,
                                     padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2,
                                     padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,
                                     padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    #Flat
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2,
                                     padding='same', activation='relu'))
    # Upsampling
    model.add(tf.keras.layers.UpSampling2D())
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2,
                                     padding='same', activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.UpSampling2D())
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2,
                                     padding='same', activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.UpSampling2D())
    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2,
                                     padding='same', activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    #Flatenning
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=512, activation='sigmoid'))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    return model

## Save model

In [5]:
def save_model(model):
    # Saving compiled model
    tf.keras.models.save_model(
        model, f'./checkpoints/{checkpoint_name}/{checkpoint_name}')

    # and its topograpy
    with open(f'./checkpoints/{checkpoint_name}/{checkpoint_name}.json', "w") as file:
        file.write(model.to_json())
        file.flush()

## Main

In [6]:
    # Loading data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Data normalization from [0,255] to [0.0,1.0]
    # and reshaping it for proper dimensions
    x_train_normalized = (x_train/255.0).reshape(x_train.shape[0], 28, 28, 1)
    x_test_normalized = (x_test/255.0).reshape(x_test.shape[0], 28, 28, 1)
    # Establish the model's topography.
    model = create_model(learning_rate)
    # Train the model on the normalized training set.
    model.fit(x=x_train_normalized, y=y_train, batch_size=batch_size,
              epochs=epochs, shuffle=True,
              validation_split=validation_split)
    # Evaluate against the test set.
    print("\n Evaluate the new model against the test set:")
    model.evaluate(x=x_test_normalized, y=y_test, batch_size=batch_size)
    save_model(model)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/40
12/12 [==============================] - 2s 148ms/step - loss: 3.6056 - accuracy: 0.1016 - val_loss: 2.8664 - val_accuracy: 0.0914
Epoch 2/40
12/12 [==============================] - 2s 133ms/step - loss: 2.6683 - accuracy: 0.0998 - val_loss: 2.4847 - val_accuracy: 0.1035
Epoch 3/40
12/12 [==============================] - 2s 132ms/step - loss: 2.4514 - accuracy: 0.0998 - val_loss: 2.3275 - val_accuracy: 0.1035
Epoch 4/40
12/12 [==============================] - 2s 131ms/step - loss: 2.3956 - accuracy: 0.1005 - val_loss: 2.3064 - val_accuracy: 0.1060
Epoch 5/40
12/12 [==============================] - 2s 133ms/step - loss: 2.3693 - accuracy: 0.1006 - val_loss: 2.3071 - val_accuracy: 0.0989
Epoch 6/40
12/12 [==============================] - 2s 131ms/step - loss: 2.3593 - accuracy: 0.1011 - val_loss: 2.3124 - val_accuracy: 0.0989
Epoch 7/40
12/12 [==============================] - 2s 132ms/step - loss: 2.3514 - 